# LightGBM para Predicción de Fallas en Mantenimiento Predictivo

Este notebook aplica LightGBM para predecir fallas usando datos reales de la tabla `faliure_probability_base` de MySQL.

Cubre:
- Carga de datos desde MySQL
- Preparación y feature engineering
- Entrenamiento de modelo LightGBM
- Evaluación y métricas
- Predicciones y análisis de resultados
- Guardar y cargar modelos

## 1. Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from mysql.connector import Error
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix,
    roc_auc_score, roc_curve, auc
)
from sklearn.preprocessing import label_binarize
import joblib
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Librerías importadas correctamente")

## 2. Carga de Datos desde MySQL

In [ ]:
# Configuración de la base de datos
DB_CONFIG = {
    'host': '127.0.0.1',
    'database': 'palantir_maintenance',
    'user': 'root',
    'password': 'admin',
    'port': 3306
}

def cargar_datos_failure_base():
    """Cargar datos de la tabla faliure_probability_base"""
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        if connection.is_connected():
            query = """
            SELECT * 
            FROM faliure_probability_base
            ORDER BY extraction_date DESC
            """
            df = pd.read_sql(query, connection)
            connection.close()
            print(f"✅ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
            return df
    except Error as e:
        print(f"❌ Error al conectar: {e}")
        return None

# Cargar datos
df = cargar_datos_failure_base()

if df is not None and len(df) > 0:
    print(f"\nColumnas disponibles: {list(df.columns)}")
    print(f"\nPrimeras filas:")
    print(df.head())
    print(f"\nInformación del DataFrame:")
    print(df.info())
    print(f"\nEstadísticas descriptivas:")
    print(df.describe())
else:
    print("⚠️ No se pudieron cargar los datos. Verifica la conexión a la base de datos.")

## 3. Análisis Exploratorio de Datos (EDA)

In [ ]:
if df is not None and len(df) > 0:
    # Verificar valores faltantes
    print("Valores faltantes por columna:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Columna': missing.index,
        'Valores Faltantes': missing.values,
        'Porcentaje': missing_pct.values
    })
    missing_df = missing_df[missing_df['Valores Faltantes'] > 0].sort_values('Valores Faltantes', ascending=False)
    
    if len(missing_df) > 0:
        print(missing_df)
    else:
        print("✅ No hay valores faltantes")
    
    # Distribución de algunas variables clave
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    if 'failure_count_365d' in df.columns:
        df['failure_count_365d'].fillna(0).hist(bins=20, ax=axes[0, 0])
        axes[0, 0].set_title('Distribución de Failure Count (365 días)')
        axes[0, 0].set_xlabel('Failure Count')
        axes[0, 0].set_ylabel('Frecuencia')
    
    if 'sensor_critical_count_30d' in df.columns:
        df['sensor_critical_count_30d'].fillna(0).hist(bins=20, ax=axes[0, 1])
        axes[0, 1].set_title('Distribución de Sensor Critical Count (30 días)')
        axes[0, 1].set_xlabel('Critical Count')
        axes[0, 1].set_ylabel('Frecuencia')
    
    if 'asset_age_days' in df.columns:
        df['asset_age_days'].fillna(0).hist(bins=20, ax=axes[1, 0])
        axes[1, 0].set_title('Distribución de Edad del Activo (días)')
        axes[1, 0].set_xlabel('Edad (días)')
        axes[1, 0].set_ylabel('Frecuencia')
    
    if 'task_total_365d' in df.columns:
        df['task_total_365d'].fillna(0).hist(bins=20, ax=axes[1, 1])
        axes[1, 1].set_title('Distribución de Tareas Totales (365 días)')
        axes[1, 1].set_xlabel('Total Tareas')
        axes[1, 1].set_ylabel('Frecuencia')
    
    plt.tight_layout()
    plt.show()

## 4. Creación de Variable Objetivo

In [ ]:
if df is not None and len(df) > 0:
    def crear_clase_falla(row):
        """
        Crear variable objetivo (clase de falla) basada en:
        - failure_count_365d: número de fallas en últimos 365 días
        - sensor_critical_count_30d: número de alertas críticas en últimos 30 días
        - failure_unresolved_count: fallas no resueltas
        
        Clases:
        0: Sin riesgo (Bajo)
        1: Riesgo bajo (Medio)
        2: Riesgo alto
        3: Riesgo crítico
        """
        failure_count = row.get('failure_count_365d', 0) or 0
        sensor_critical = row.get('sensor_critical_count_30d', 0) or 0
        unresolved = row.get('failure_unresolved_count', 0) or 0
        
        # Lógica de clasificación
        score = 0
        
        # Puntos por fallas
        if failure_count >= 5:
            score += 3
        elif failure_count >= 3:
            score += 2
        elif failure_count >= 1:
            score += 1
        
        # Puntos por sensores críticos
        if sensor_critical >= 10:
            score += 3
        elif sensor_critical >= 5:
            score += 2
        elif sensor_critical >= 2:
            score += 1
        
        # Puntos por fallas no resueltas
        if unresolved >= 3:
            score += 2
        elif unresolved >= 1:
            score += 1
        
        # Clasificar según score total
        if score >= 6:
            return 3  # Crítico
        elif score >= 4:
            return 2  # Alto
        elif score >= 2:
            return 1  # Medio
        else:
            return 0  # Bajo
    
    # Aplicar función
    df['clase_falla'] = df.apply(crear_clase_falla, axis=1)
    
    # Ver distribución de clases
    print("Distribución de clases de falla:")
    clase_dist = df['clase_falla'].value_counts().sort_index()
    clase_labels = ['Bajo', 'Medio', 'Alto', 'Crítico']
    
    for idx, count in clase_dist.items():
        label = clase_labels[idx] if idx < len(clase_labels) else f'Clase {idx}'
        pct = (count / len(df)) * 100
        print(f"  {label}: {count} ({pct:.2f}%)")
    
    # Visualizar distribución
    plt.figure(figsize=(10, 6))
    clase_dist.plot(kind='bar', color=['green', 'yellow', 'orange', 'red'])
    plt.title('Distribución de Clases de Falla')
    plt.xlabel('Clase de Falla')
    plt.ylabel('Frecuencia')
    plt.xticks(range(len(clase_dist)), [clase_labels[i] if i < len(clase_labels) else f'Clase {i}' for i in clase_dist.index], rotation=0)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Variable objetivo creada: {df['clase_falla'].nunique()} clases únicas")

## 5. Selección y Preparación de Características

In [ ]:
if df is not None and 'clase_falla' in df.columns:
    # Seleccionar características relevantes
    feature_columns = [
        # Información del activo
        'asset_age_days',
        
        # Características de sensores (30 días)
        'sensor_total_readings_30d',
        'sensor_warning_count_30d',
        'sensor_critical_count_30d',
        'sensor_avg_normal_value',
        'sensor_avg_warning_value',
        'sensor_avg_critical_value',
        'sensor_max_value',
        'sensor_min_value',
        'sensor_std_value',
        
        # Características de fallas (365 días)
        'failure_count_365d',
        'failure_critical_count',
        'failure_high_count',
        'failure_medium_count',
        'failure_low_count',
        'failure_avg_downtime',
        'failure_total_downtime',
        'failure_unresolved_count',
        'days_since_last_failure',
        
        # Características de tareas de mantenimiento (365 días)
        'task_total_365d',
        'task_completed_count',
        'task_in_progress_count',
        'task_pending_count',
        'task_avg_estimated_hours',
        'task_avg_actual_hours',
        'task_total_hours',
        'days_since_last_task',
        
        # Características de órdenes de mantenimiento (365 días)
        'order_total_365d',
        'order_preventive_count',
        'order_corrective_count',
        'order_emergency_count',
        'order_completed_count',
        'order_avg_estimated_cost',
        'order_avg_actual_cost',
        'order_total_actual_cost',
        'days_since_last_order'
    ]
    
    # Filtrar columnas que existen en el DataFrame
    available_features = [col for col in feature_columns if col in df.columns]
    
    print(f"Características seleccionadas: {len(available_features)}")
    print(f"\nCaracterísticas: {available_features}")
    
    # Preparar datos
    X = df[available_features].copy()
    y = df['clase_falla'].copy()
    
    # Manejar valores faltantes
    print(f"\nValores faltantes antes de limpieza: {X.isnull().sum().sum()}")
    
    # Llenar valores faltantes con 0 (asumiendo que NaN significa "sin datos")
    X = X.fillna(0)
    
    # Reemplazar infinitos con 0
    X = X.replace([np.inf, -np.inf], 0)
    
    print(f"Valores faltantes después de limpieza: {X.isnull().sum().sum()}")
    print(f"\nForma de X: {X.shape}")
    print(f"Forma de y: {y.shape}")
    print(f"\nTipos de datos:")
    print(X.dtypes.value_counts())
    
    # Verificar que tenemos suficientes datos
    if len(X) < 50:
        print("⚠️ ADVERTENCIA: Muy pocos datos para entrenar un modelo robusto")
    else:
        print(f"✅ Datos suficientes para entrenamiento: {len(X)} muestras")

## 6. División de Datos en Train y Test

In [ ]:
if 'X' in locals() and 'y' in locals():
    # Dividir datos (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=42, 
        stratify=y  # Mantener proporción de clases
    )
    
    print(f"Datos de entrenamiento: {X_train.shape}")
    print(f"Datos de prueba: {X_test.shape}")
    
    print(f"\nDistribución de clases en entrenamiento:")
    print(y_train.value_counts().sort_index())
    
    print(f"\nDistribución de clases en prueba:")
    print(y_test.value_counts().sort_index())
    
    # Verificar balanceo de clases
    train_dist = y_train.value_counts().sort_index() / len(y_train)
    test_dist = y_test.value_counts().sort_index() / len(y_test)
    
    print(f"\nProporciones en entrenamiento:")
    for idx, pct in train_dist.items():
        label = clase_labels[idx] if idx < len(clase_labels) else f'Clase {idx}'
        print(f"  {label}: {pct:.2%}")
    
    print(f"\nProporciones en prueba:")
    for idx, pct in test_dist.items():
        label = clase_labels[idx] if idx < len(clase_labels) else f'Clase {idx}'
        print(f"  {label}: {pct:.2%}")
else:
    print("⚠️ No hay datos preparados")

## 7. Entrenamiento del Modelo LightGBM

In [ ]:
if 'X_train' in locals() and 'y_train' in locals():
    # Configurar parámetros de LightGBM
    lgbm_params = {
        'objective': 'multiclass',
        'num_class': len(y_train.unique()),
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'min_child_samples': 20,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'random_state': 42,
        'verbose': -1
    }
    
    print("Parámetros del modelo LightGBM:")
    for key, value in lgbm_params.items():
        print(f"  {key}: {value}")
    
    # Crear y entrenar modelo
    print("\n🚀 Entrenando modelo LightGBM...")
    
    lgbm_model = LGBMClassifier(**lgbm_params)
    
    # Entrenar con validación temprana
    lgbm_model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_names=['train', 'valid'],
        callbacks=[
            lgbm_model.early_stopping(stopping_rounds=20, verbose=True),
            lgbm_model.log_evaluation(period=10)
        ]
    )
    
    print("\n✅ Modelo entrenado correctamente")
    
    # Mostrar mejor iteración
    if hasattr(lgbm_model, 'best_iteration_'):
        print(f"Mejor iteración: {lgbm_model.best_iteration_}")
else:
    print("⚠️ No hay datos de entrenamiento")

## 8. Predicciones y Evaluación

In [ ]:
if 'lgbm_model' in locals() and 'X_test' in locals():
    # Hacer predicciones
    y_pred = lgbm_model.predict(X_test)
    y_pred_proba = lgbm_model.predict_proba(X_test)
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    print("=== MÉTRICAS DEL MODELO ===")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    
    # Classification Report
    print("\n=== CLASSIFICATION REPORT ===")
    print(classification_report(y_test, y_pred, target_names=clase_labels))
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=clase_labels, yticklabels=clase_labels)
    plt.title('Matriz de Confusión - LightGBM')
    plt.ylabel('Verdadero')
    plt.xlabel('Predicho')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No hay modelo entrenado")

## 9. Importancia de Características

In [ ]:
if 'lgbm_model' in locals() and 'X_train' in locals():
    # Obtener importancia de características
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': lgbm_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("Top 15 características más importantes:")
    print(feature_importance.head(15))
    
    # Visualizar
    plt.figure(figsize=(12, 10))
    sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
    plt.title('Top 15 Características Más Importantes - LightGBM')
    plt.xlabel('Importancia')
    plt.tight_layout()
    plt.show()
    
    # Análisis de características más importantes
    print("\n=== ANÁLISIS DE CARACTERÍSTICAS ===")
    top_5 = feature_importance.head(5)
    print("\nLas 5 características más importantes son:")
    for idx, row in top_5.iterrows():
        print(f"  {row['feature']}: {row['importance']:.2f}")

## 10. Análisis de Predicciones

In [ ]:
if 'y_pred' in locals() and 'y_test' in locals():
    # Crear DataFrame con predicciones y valores reales
    resultados_df = pd.DataFrame({
        'Real': y_test.values,
        'Predicho': y_pred,
        'Correcto': (y_test.values == y_pred)
    })
    
    # Agregar probabilidades
    if 'y_pred_proba' in locals():
        for i in range(y_pred_proba.shape[1]):
            resultados_df[f'Prob_Clase_{i}'] = y_pred_proba[:, i]
    
    # Análisis por clase
    print("=== ANÁLISIS DE PREDICCIONES POR CLASE ===")
    for clase in sorted(y_test.unique()):
        clase_nombre = clase_labels[clase] if clase < len(clase_labels) else f'Clase {clase}'
        mask = resultados_df['Real'] == clase
        total = mask.sum()
        correctos = resultados_df[mask]['Correcto'].sum()
        accuracy_clase = correctos / total if total > 0 else 0
        
        print(f"\n{clase_nombre}:")
        print(f"  Total muestras: {total}")
        print(f"  Predicciones correctas: {correctos}")
        print(f"  Accuracy: {accuracy_clase:.2%}")
    
    # Visualizar distribución de probabilidades
    if 'y_pred_proba' in locals():
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        for i, clase in enumerate(sorted(y_test.unique())):
            if i < 4:
                ax = axes[i // 2, i % 2]
                clase_nombre = clase_labels[clase] if clase < len(clase_labels) else f'Clase {clase}'
                
                mask = resultados_df['Real'] == clase
                ax.hist(resultados_df[mask][f'Prob_Clase_{clase}'], bins=20, alpha=0.7, label='Correcto')
                
                mask_incorrecto = mask & ~resultados_df['Correcto']
                if mask_incorrecto.sum() > 0:
                    ax.hist(resultados_df[mask_incorrecto][f'Prob_Clase_{clase}'], bins=20, alpha=0.7, label='Incorrecto')
                
                ax.set_title(f'Distribución de Probabilidades - {clase_nombre}')
                ax.set_xlabel('Probabilidad')
                ax.set_ylabel('Frecuencia')
                ax.legend()
                ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print("\nPrimeras 10 predicciones:")
    print(resultados_df.head(10))

## 11. Guardar y Cargar el Modelo

In [ ]:
if 'lgbm_model' in locals():
    # Guardar modelo
    modelo_path = 'lightgbm_failure_prediction_model.pkl'
    joblib.dump(lgbm_model, modelo_path)
    print(f"✅ Modelo guardado en: {modelo_path}")
    
    # Guardar información de características
    feature_info = {
        'feature_names': list(X_train.columns),
        'num_classes': len(y_train.unique()),
        'class_labels': clase_labels
    }
    
    import json
    with open('model_feature_info.json', 'w') as f:
        json.dump(feature_info, f, indent=2)
    
    print("✅ Información de características guardada en: model_feature_info.json")
    
    # Ejemplo de cómo cargar el modelo
    print("\n=== EJEMPLO DE CARGA DEL MODELO ===")
    print("# Para cargar el modelo en el futuro:")
    print("# loaded_model = joblib.load('lightgbm_failure_prediction_model.pkl')")
    print("# predictions = loaded_model.predict(new_data)")
    
    # Verificar carga
    loaded_model = joblib.load(modelo_path)
    test_pred = loaded_model.predict(X_test.head(5))
    print(f"\n✅ Modelo cargado correctamente. Predicción de prueba: {test_pred}")
else:
    print("⚠️ No hay modelo para guardar")

## 12. Predicción en Nuevos Datos

In [ ]:
# Función para hacer predicciones en nuevos datos
def predecir_fallas(nuevos_datos, modelo, feature_names):
    """
    Hacer predicciones de fallas en nuevos datos
    
    Parámetros:
    - nuevos_datos: DataFrame con las características
    - modelo: Modelo LightGBM entrenado
    - feature_names: Lista de nombres de características esperadas
    
    Retorna:
    - predicciones: Array con las clases predichas
    - probabilidades: Array con las probabilidades por clase
    """
    # Asegurar que tenemos las columnas correctas
    datos_preparados = nuevos_datos[feature_names].copy()
    
    # Llenar valores faltantes
    datos_preparados = datos_preparados.fillna(0)
    datos_preparados = datos_preparados.replace([np.inf, -np.inf], 0)
    
    # Hacer predicciones
    predicciones = modelo.predict(datos_preparados)
    probabilidades = modelo.predict_proba(datos_preparados)
    
    return predicciones, probabilidades

# Ejemplo de uso
if 'lgbm_model' in locals() and 'X_test' in locals():
    print("=== EJEMPLO DE PREDICCIÓN EN NUEVOS DATOS ===")
    
    # Usar algunas muestras de test como ejemplo
    ejemplo_datos = X_test.head(3)
    
    pred_ejemplo, prob_ejemplo = predecir_fallas(
        ejemplo_datos, 
        lgbm_model, 
        list(X_train.columns)
    )
    
    print(f"\nPredicciones: {pred_ejemplo}")
    print(f"\nProbabilidades:")
    for i, (pred, prob) in enumerate(zip(pred_ejemplo, prob_ejemplo)):
        clase_nombre = clase_labels[pred] if pred < len(clase_labels) else f'Clase {pred}'
        print(f"\nMuestra {i+1}:")
        print(f"  Clase predicha: {clase_nombre} ({pred})")
        print(f"  Probabilidades:")
        for j, p in enumerate(prob):
            clase_p = clase_labels[j] if j < len(clase_labels) else f'Clase {j}'
            print(f"    {clase_p}: {p:.4f}")
else:
    print("⚠️ No hay modelo disponible para hacer predicciones")

## Resumen

En este notebook hemos aprendido:
1. ✅ Carga de datos desde MySQL (tabla `faliure_probability_base`)
2. ✅ Análisis exploratorio de datos
3. ✅ Creación de variable objetivo (clases de falla)
4. ✅ Preparación y selección de características
5. ✅ Entrenamiento de modelo LightGBM
6. ✅ Evaluación con múltiples métricas
7. ✅ Análisis de importancia de características
8. ✅ Análisis detallado de predicciones
9. ✅ Guardar y cargar modelos
10. ✅ Función para predecir en nuevos datos

**El modelo está listo para ser usado en producción para predecir fallas en activos de mantenimiento.**